In [ ]:
from pathlib import Path
import numpy as np
import soundfile
import matplotlib.pyplot as plt
import librosa
from pprint import pprint
import scipy.ndimage
import scipy.signal
import logging
import itertools
import scipy.sparse
import datetime
import os
import pickle
import time

from unmixdb import UnmixDB
from abcdj import ABCDJ
import activation_learner, carve, plot, param_estimator, modular_nmf

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
FS = 22050
T = np.linspace(0, 10, 10 * FS)
inputs = [
    0.0001 * np.sin(2 * np.pi * 200 * T**2),
    np.sin(2 * np.pi * 200 * T**2) * np.linspace(0, 1, len(T)),
]
win_size = 1
hop_size = 0.25
BETA = 0
NMELS = 256
# stop conditions
DLOSS_MIN = 1e-8
LOSS_MIN = -np.inf
ITER_MAX = 500
# logging
PLOT_NMF_EVERY = 50
DIVERGENCE = modular_nmf.BetaDivergence(0)
PENALTIES = [
    # (modular_nmf.L1(), 9643.097400544748),
    # (modular_nmf.SmoothGain(), 1572.5437219589044),
    # (modular_nmf.VirtanenTemporalContinuity(), 1)
]

In [ ]:
learner = activation_learner.ActivationLearner(
    inputs,
    fs=FS,
    n_mels=NMELS,
    win_size=win_size,
    hop_size=hop_size,
    divergence=DIVERGENCE,
    penalties=PENALTIES,
)
last_loss = np.inf
for i in itertools.count():
    # TODO: get all components of loss
    loss,_ = learner.iterate()

    if i > ITER_MAX:
        print(f"Stopped at NMF iteration={i} loss={loss}")
        break
    if i % PLOT_NMF_EVERY == 0:
        print(f"NMF iteration={i} loss={loss:.2e}")
        plot.plot_nmf(learner)
        plt.show()
plot.plot_nmf(learner)
plt.show()

plt.close("all")

In [ ]:
tau = np.arange(0, learner.V.shape[1]) * hop_size

results = {}
# estimate gain
results["est_gain"] = {}
for estimator in param_estimator.GainEstimator:
    print(f"Estimating gain with method {estimator}")
    est_gain = estimator.value(learner)

    fig = plt.figure()
    plot.plot_gain(tau, est_gain)
    plt.show()
    results["est_gain"][str(estimator)] = est_gain

# estimate warp
results["est_warp"] = {}
for estimator in param_estimator.WarpEstimator:
    print(f"Estimating warp with method {estimator}")
    est_warp = estimator.value(learner)

    fig = plt.figure()
    plot.plot_warp(tau, est_warp)
    plt.show()

    results["est_warp"][str(estimator)] = est_warp